# 호텔 챗봇

이 노트북에서는 호텔 챗봇 문제를 다룹니다. 어떻게 하면 호텔 프론트 데스크에서 더 나은 고객 서비스를 제공할 수 있을까요? 간단한 방법 중 하나는 호텔에서 경험할 수 있는 일들에 관한 간단한 질문에 대답할 수 있는 FAQ 채팅 로봇을 갖는 것입니다. 챗봇을 갖는 것에는 많은 장점이 있습니다.

1. 호텔의 매력을 높이고 정보 처리량을 늘립니다.
2. 호텔에 대한 질문을 데이터 테이블 형식으로 수집할 수 있는 방법을 만듭니다.

이 노트북에서는 두 가지 모델을 살펴볼 것입니다. 코사인 유사성 및 doc2vec 모델입니다.


## 1. 챗봇에 지식 기반 추가

챗봇의 대화 능력은 사용할 수 있는 데이터에 의해 정의됩니다. Ques.txt 파일과 ans.txt 파일에서 질문과 답변을 살펴보십시오. 이 챗봇은 기본적으로 질문에 대하여 질문 은행과 코사인 유사성을 확인하여 답을 찾으려고 노력할 것입니다.

In [1]:
import nltk # 텍스트 데이터를 처리
import numpy as np # 말뭉치를 배열로 표현
import random 
import operator
import string # 표준 파이썬 문자열을 처리
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!

In [2]:
import os

filepath=os.getcwd()+'/data/[Dataset] Module27 (ans).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data_ans=corpus.read()
print (raw_data_ans)

filepath=os.getcwd()+'/data/[Dataset] Module27(ques).txt'
corpus=open(filepath,'r',errors = 'ignore')
raw_data=corpus.read()
print (raw_data)

200$ per night is the price for a basic suite.

This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.

Breakfast is served from 7 AM to 10 AM.

The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before. 

The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.

There are 43 rooms in this hotel including one pent house suite.

We offer 4 types of rooms: basic, mid-level, premium and penthouse.

This hotel is called Vance Hotel.

Yes, room service is available 24 hrs.

To call room service, please dial '0' using the phone in your room.

Yes we have one restaurant currently called 'Rouge'.

There are 12 floors in the hotel.

300$ per night is the standard price for a mid-level suite.

500$ per night is the price for a premium suite.

Yes, we

####  소문자로 변환

모든 텍스트를 먼저 소문자로 변환합니다. 결과가 완료되면 검사해야 합니다.

In [3]:
raw_data = raw_data.lower() # 소문자로 변환
print (raw_data)

what is the price of one night stay in basic suite?

how old is this establishment?

what time is breakfast served?

what is the breakfast menu?

what is the history behind this hotel or establishment?

how many rooms are there in this hotel?

what are the types of rooms or suites offered by the hotel?

what is the name of the hotel?

is room service served 24 hours?

how do i call for room service?

are there any restaurants in the hotel?

how many floors are there in the hotel?

what is the price of one night stay at the mid-level suite?

what is the price of one night stay at the premium suite?

do you have tuxedo services?

do you have a laundry service?

what time do the restaurants open for dinner?

what are the near by tourist attractions?

is there a spa in the hotel?

is there anywhere i can get a massage?

what time is the check in?

what time is the check out?

do you offer handicapped rooms?

is parking available at the hotel?

can i reserve a parking lot?

what are the rec

####  세분화, 표제어 추출,  단어 토큰화

모듈 26 코사인 유사성 노트북에서 배운 방법을 참조하여 데이터를 세분화, 토큰화, 표제어 추출로 전처리 진행합니다.

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw_data)# 문서를 문장 목록으로 변환

print(sent_tokens)

['what is the price of one night stay in basic suite?', 'how old is this establishment?', 'what time is breakfast served?', 'what is the breakfast menu?', 'what is the history behind this hotel or establishment?', 'how many rooms are there in this hotel?', 'what are the types of rooms or suites offered by the hotel?', 'what is the name of the hotel?', 'is room service served 24 hours?', 'how do i call for room service?', 'are there any restaurants in the hotel?', 'how many floors are there in the hotel?', 'what is the price of one night stay at the mid-level suite?', 'what is the price of one night stay at the premium suite?', 'do you have tuxedo services?', 'do you have a laundry service?', 'what time do the restaurants open for dinner?', 'what are the near by tourist attractions?', 'is there a spa in the hotel?', 'is there anywhere i can get a massage?', 'what time is the check in?', 'what time is the check out?', 'do you offer handicapped rooms?', 'is parking available at the hotel?

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)# 문서를 문장 목록으로 변환
print(sent_tokens_ans)

['200$ per night is the price for a basic suite.', 'This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.', 'Breakfast is served from 7 AM to 10 AM.', 'The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before.', 'The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.', 'There are 43 rooms in this hotel including one pent house suite.', 'We offer 4 types of rooms: basic, mid-level, premium and penthouse.', 'This hotel is called Vance Hotel.', 'Yes, room service is available 24 hrs.', "To call room service, please dial '0' using the phone in your room.", "Yes we have one restaurant currently called 'Rouge'.", 'There are 12 floors in the hotel.', '300$ per night is the standard price for a mid-level suite.', '500$ per night is the price fo

In [6]:
res = {sent_tokens[i]: sent_tokens_ans[i] for i in range(len(sent_tokens))} 
print(res)

{'what is the price of one night stay in basic suite?': '200$ per night is the price for a basic suite.', 'how old is this establishment?': 'This establishment was constructed and inaugurated by John S. on the 23rd of September 1965.', 'what time is breakfast served?': 'Breakfast is served from 7 AM to 10 AM.', 'what is the breakfast menu?': 'The breakfast menu is decided as per the head chefs decision on the night before; kindly contact hotel staff for information about the menu on the night before.', 'what is the history behind this hotel or establishment?': 'The Vance Hotel is a singular establishment designed and constructed by Lindsey Vance in 1949; it has hosted many dignitaries and government officials over the years.', 'how many rooms are there in this hotel?': 'There are 43 rooms in this hotel including one pent house suite.', 'what are the types of rooms or suites offered by the hotel?': 'We offer 4 types of rooms: basic, mid-level, premium and penthouse.', 'what is the name 

In [11]:
word_tokens = nltk.word_tokenize(raw_data)# 문서를 단어 목록으로 변환
print (word_tokens)

['what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'in', 'basic', 'suite', '?', 'how', 'old', 'is', 'this', 'establishment', '?', 'what', 'time', 'is', 'breakfast', 'served', '?', 'what', 'is', 'the', 'breakfast', 'menu', '?', 'what', 'is', 'the', 'history', 'behind', 'this', 'hotel', 'or', 'establishment', '?', 'how', 'many', 'rooms', 'are', 'there', 'in', 'this', 'hotel', '?', 'what', 'are', 'the', 'types', 'of', 'rooms', 'or', 'suites', 'offered', 'by', 'the', 'hotel', '?', 'what', 'is', 'the', 'name', 'of', 'the', 'hotel', '?', 'is', 'room', 'service', 'served', '24', 'hours', '?', 'how', 'do', 'i', 'call', 'for', 'room', 'service', '?', 'are', 'there', 'any', 'restaurants', 'in', 'the', 'hotel', '?', 'how', 'many', 'floors', 'are', 'there', 'in', 'the', 'hotel', '?', 'what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'at', 'the', 'mid-level', 'suite', '?', 'what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'at', 'the', 'premium', 'suite', '?', 'do', 'yo

In [7]:
lemmer = nltk.stem.WordNetLemmatizer() # lemmer 클래스를 초기화합니다. WordNet은 NLTK에 포함된 의미 론적 영어 사전입니다
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [8]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) 

## 2. 챗봇 기능 추가 - 코사인 유사성

In [9]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    for word in sentence.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in GREETING_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(GREETING_RESPONSES) # Greeting_Response로 답장합니다.

챗봇의 기능은 챗봇을 실행하기 위한 루프를 만듦으로써 이루어집니다. 아래 기능을 살펴봅니다. 함수의 각 줄은 중요한 단계를 수행하기 위해 다른 함수를 호출하기 때문에 중요합니다. 'response' 함수는 챗봇이 어떻게 행동하는지에 대한 작동 방식을 담당합니다.

In [10]:
def response(user_response):
    
    robo_response='' # 문자열을 포함하도록 변수를 초기화
    sent_tokens.append(user_response) # sent_messages에 사용자 응답 추가
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens) # tfidf 값 가져오기
    vals = cosine_similarity(tfidf[-1], tfidf) # 코사인 유사성 값 가져오기
    idx=vals.argsort()[0][-2] 
    flat = vals.flatten() 
    flat.sort() # 오름차순으로 정렬
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response, vals
    else:
        robo_response = robo_response+res[sent_tokens[idx]]
        return robo_response, vals

마지막으로 챗봇 인터페이스를 만들고 이를 중심으로 페르소나를 만들어 봅시다. 우리가 챗봇을 '제인'이라고 부르고 코사인 유사성을 사용하여 질문과 유사한 FAQ를 찾아 대답하도록 합시다.

In [12]:
flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= response(user_response)
                print(resp[0], )
                sent_tokens.remove(user_response)
                resp_l = resp[1].tolist()
                resp_l[0].pop()
                print(' (With similarity of ',max(resp_l[0]),')')

    else:
        flag=False
        print("Jane: Bye! take care..")
       

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Jane: 

d:\Intel\NLP_lecture\nlp\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
d:\Intel\NLP_lecture\nlp\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


This hotel is called Vance Hotel.
 (With similarity of  0.3893470080671374 )
Jane: 500$ per night is the price for a premium suite.
 (With similarity of  0.5883971074889468 )
Jane: Bye! take care..


## 3. Doc2vec를 이용한 챗봇 기능

챗봇을 만들기 위한 한 가지 유형의 모델을 더 다룰 것입니다. 코사인 유사성 모델 알고리즘은 두 문장 사이의 유사성을 찾는데 사용됩니다. 하지만 이제 신경망을 사용해서 이 문제를 해결해 보려합니다. 살펴보도록 하겠습니다.

Doc2Vec은 기본적으로 문서에서 벡터를 생성하는 신경망 기반 모델입니다. Doc2vec을 이해하기 위해서는 word2vec도 이해해야 합니다.

#### word2vec란?
이는 삽입 단어를 생성하는 모델이며, 여기서는 텍스트의 큰 말뭉치를 입력으로 받고 일반적으로 수백 개 차원의 벡터 공간을 생성합니다. 

2013년 9월과 10월 사이에 구글의 연구팀에 의해 두 개의 논문에 소개되었습니다. Word2Vec의 기본 가정은 유사한 맥락을 공유하는 두 단어가 유사한 의미를 공유하며 결과적으로 모델에서 유사한 벡터 표현을 공유한다는 것입니다.

예를 들어, "은행", "화폐", "계좌"는 "달러", "대출", "신용"과 같은 유사한 주변 단어와 함께 종종 사용되며, 따라서 Word2Vec에 따르면 이들은 유사한 벡터 표현을 공유합니다.

![Image](img1.png)

#### doc2vec 란?

doc2vec의 목적은 말뭉치의 모든 단어에 대한 특징 벡터를 계산하는 word2vec와 달리 문장/단락/문서의 수치 표현을 생성하는 것입니다. doc2vec은 말뭉치의 모든 문서에 대한 특징 벡터를 계산합니다. doc2vec에 의해 생성된 벡터는 문장/단락/문서 간의 유사성 찾기와 같은 작업에 사용될 수 있습니다.

<strong> doc2vec의 속성을 사용하여 우리만의 유사성 모델을 만들 것입니다.</strong>


관련 라이브러리를 가져오는 것으로 시작하겠습니다.

In [13]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

첫 번째 주요 목표는 데이터에 태그를 지정하는 것입니다. doc2vec 모델은 데이터를 효과적으로 사용하기 위해 태그를 지정해야 합니다. 다음은 doc2vec에 대한 [시작 코드](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) 좋은 학습 링크입니다. 다음 [링크](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) 도 유용하게 사용할 수 있으며 읽을 것을 권장합니다.

In [15]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens)]

In [16]:
max_epochs = 100
vec_size = 20 # 벡터가 더 크려면 이것을 증가시키세요. 이것은 더 많은 차이를 의미합니다. 
alpha = 0.025


다음 단계는 모델을 훈련시키는 것입니다. 이전처럼 훈련 과정을 실행하기 위해 `model.train` 함수를 사용하게 될 것입니다. Doc2vec 모델을 훈련하는 방법에 대한 자세한 정보는 [문서](https://radimrehurek.com/gensim/models/doc2vec.html) 를 참조하십시오.

In [17]:
model = Doc2Vec(size =vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # 여기에 사용할 태그가 지정된 데이터
                total_examples=model.corpus_count,
                epochs=100)
    # 학습 비율을 줄입니다
    model.alpha -= 0.0002
    # 학습 비율을 고정합니다.
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


d:\Intel\NLP_lecture\nlp\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### doc2vec 모델 평가

In [18]:
from gensim.models.doc2vec import Doc2Vec
model= Doc2Vec.load("d2v.model")

In [19]:
test_data = word_tokenize("How much is the price?".lower())

`model.infer_vector` 함수를 사용하여 문서와 관련된 벡터를 추론할 수 있습니다. 그런 다음`most_similar` 함수를 사용하여 우리가 만든 벡터와 가장 유사한 벡터를 찾을 수 있습니다. 결과는 어떻습니까?

In [20]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-0.12124646 -0.29389238 -0.5955331  -0.03794789  0.03689046 -0.44410917
 -0.10979072  0.14911555 -0.4933019   0.6537895   0.40305847  0.15220141
  0.07170007 -0.45990878 -0.59167033  0.24461882 -0.21134788  0.15527983
  0.01040273  0.05213068]


In [21]:
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('0', 0.7205744385719299), ('2', 0.6770764589309692), ('31', 0.6648393869400024), ('25', 0.6648154258728027)]


In [22]:
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]


TaggedDocument(words=['what', 'is', 'the', 'price', 'of', 'one', 'night', 'stay', 'in', 'basic', 'suite', '?'], tags=['0'])

이전 코드 블록의 출력에서 이 모델이 생각했던 것 만큼 효과적이지 않다는 것이 분명합니다. 이 모델이 성공하기를 기대했지만 실패한 이유가 있습니까?

자세한 내용을 보려면 이 [링크](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) 를 클릭하십시오. 이 문제의 요지는 다음과 같습니다.

> Doc2Vec는 장난감 크기(toy-size)의 데이터셋에서 좋은 결과를 얻을 수 없으므로, 더 많은 데이터를 사용하기 전까지는 의미 있는 결과를 기대해서는 안 됩니다.

### 사전 학습된 doc2vec 사용

따라서 인터넷에서 다운로드한 사전 검증된 모델을 사용하여 사용 사례를 최적화해 보겠습니다. The 우리가 취득 한 모델은 관련 언론 뉴스에 대한 훈련을 받았으며  [여기](https://github.com/jhlau/doc2vec) 서 다운로드 할 수 있습니다.

모델을 로드하고 다시 평가해 보겠습니다.

In [24]:
model= Doc2Vec.load(os.getcwd()+"/models/doc2vec.bin")

In [25]:
test_data = word_tokenize("How much is the price?".lower())

In [26]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 3.45155969e-02 -2.55955383e-02  4.25544195e-03  2.34726537e-03
 -1.19248573e-02  7.07957894e-02 -8.46000984e-02 -6.44249842e-02
 -2.17548925e-02  5.78956818e-03  8.65090452e-03  8.48189592e-02
 -4.07059751e-02 -5.77710159e-02 -9.85386819e-02  3.22032608e-02
 -7.33430237e-02  3.49471532e-02 -6.27333298e-02 -9.13093686e-02
 -9.96336117e-02 -4.01511937e-02  2.40963623e-02  7.39931613e-02
 -4.45079803e-03 -1.05832636e-01 -2.48195473e-02 -4.52671051e-02
 -2.77641769e-02  1.36740068e-02 -8.54790062e-02 -7.99114350e-03
 -2.90457327e-02  2.42306143e-02  5.64891398e-02  2.46598702e-02
  1.67198498e-02  9.84575367e-04 -2.83648744e-02 -1.00909658e-01
  2.37279143e-02  2.37233248e-02  5.68677001e-02  1.33365523e-02
 -2.28467193e-02 -2.77601704e-02  2.67025945e-03 -4.98644561e-02
 -1.82799883e-02  5.40140085e-02 -6.93371817e-02 -4.73205820e-02
 -4.05686796e-02 -3.51031534e-02  6.74198195e-03 -4.86985855e-02
  1.31985964e-02  3.77716720e-02  4.09564190e-02 -7.57795870e-02
  6.94855377e-02

In [27]:
for i in sent_tokens:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

what is the price of one night stay in basic suite?
[[0.58686936]]
how old is this establishment?
[[0.48568475]]
what time is breakfast served?
[[0.44974625]]
what is the breakfast menu?
[[0.52232504]]
what is the history behind this hotel or establishment?
[[0.42273846]]
how many rooms are there in this hotel?
[[0.5166798]]
what are the types of rooms or suites offered by the hotel?
[[0.43157318]]
what is the name of the hotel?
[[0.5521878]]
is room service served 24 hours?
[[0.34110713]]
how do i call for room service?
[[0.5226105]]
are there any restaurants in the hotel?
[[0.5191803]]
how many floors are there in the hotel?
[[0.46610382]]
what is the price of one night stay at the mid-level suite?
[[0.5418877]]
what is the price of one night stay at the premium suite?
[[0.59164524]]
do you have tuxedo services?
[[0.45656478]]
do you have a laundry service?
[[0.5198914]]
what time do the restaurants open for dinner?
[[0.5032941]]
what are the near by tourist attractions?
[[0.53591764

In [28]:
def calc_prob(v1, q):
    probs = dict()
    for i in q:
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]
    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))
    
    return list(sorted_d.items())[0]

In [29]:
calc_prob(v1, sent_tokens)

('why do the costs vary from day to day?', 0.6434249)

이 모델이 더 효과적인 것처럼 보이기 때문에, 우리의 챗봇에 통합해 봅시다.

In [30]:
flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens)
                print(res[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("Jane: Bye! take care..")

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Jane: Yes, we do have a spa; however, it is closed from 7pm onwards.
 (With similarity of  0.719463 )
Jane: Check in is at anytime between 10am and 1pm.
 (With similarity of  0.7074312 )
Jane: 500$ per night is the price for a premium suite.
 (With similarity of  0.7082869 )
Jane: Bye! take care..


**두 모델의 성능** 을 관찰한 후 다음과 같은 몇 가지 명확한 결론을 내릴 수 있습니다:

1. Doc2vec 모델은 단어 간의 관계를 이해하기 위해 더 많은 데이터가 필요합니다. 그리고 사전 훈련된 모델을 사용한 후에도 코사인 유사성 모델에 비해 모델의 응답의 품질이 아직 부족합니다.
2. 코사인 유사성 모델은 더 작고 잘 정의된 데이터 세트에서 더 잘 작동합니다. 이는 몇 가지 간단한 질문을 효과적으로 해결할 수 있지만 컨텍스트를 필요로 하는 복잡한 질문은 해결할 수 없다는 의미입니다.